In [ ]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
X_res, y_res = ros.fit_resample(X_train, y_train)

print("Before resampling:", X_train.shape, "fires:", sum(y_train))
print("After resampling:", X_res.shape, "fires:", sum(y_res))

rf = RandomForestClassifier(n_estimators=500, random_state=42, max_depth=15)
rf.fit(X_res, y_res)
y_prob = rf.predict_proba(X_test)[:, 1]

print("ROC-AUC:", roc_auc_score(y_test, y_prob))
print(classification_report(y_test, (y_prob >= 0.3).astype(int)))

In [ ]:
import numpy as np

# Use last available date in weather data
latest_date = weather["date"].max()
latest_weather = weather[weather["date"] == latest_date][["cell_id"] + feature_cols]

# Predict fire probabilities
latest_weather["fire_risk"] = rf.predict_proba(latest_weather[feature_cols])[:, 1]

latest_weather.to_csv("predictions.csv", index=False)
print("Predictions saved for", latest_date, "with shape", latest_weather.shape)
print(latest_weather.head())


In [ ]:
import pandas as pd

# Load weather + grid
weather = pd.read_csv("weather.csv", parse_dates=["time"])
grid = pd.read_csv("grid.csv")

# Last available date in weather data
latest_date = weather["time"].max()
print("Latest date:", latest_date)

# Filter latest weather
latest_weather = weather[weather["time"] == latest_date].copy()

# Select features
feature_cols = [
    "temperature_2m_max",
    "temperature_2m_min",
    "precipitation_sum",
    "windspeed_10m_max",
    "relative_humidity_2m_mean"
]

# Predict fire risk
latest_weather["fire_risk"] = rf.predict_proba(latest_weather[feature_cols])[:, 1]

# Merge centroids for plotting
pred_map = latest_weather.merge(
    grid[["cell_id", "centroid_lon", "centroid_lat"]],
    on="cell_id"
)

pred_map.to_csv("predictions.csv", index=False)
print("✅ Saved predictions.csv with shape:", pred_map.shape)
pred_map.head()